In [1]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.WARNING, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

{
  "shell_port": 61715,
  "iopub_port": 61716,
  "stdin_port": 61717,
  "control_port": 61718,
  "hb_port": 61719,
  "ip": "127.0.0.1",
  "key": "937a97ea-6109a469cb73c124c60647d5",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-82d25ff3-98ac-4ccc-a045-652dc0422a4d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned OFF


In [9]:
from itertools import combinations
from scipy.special import factorial

In [58]:
value_t = {
    frozenset(): 0,
    frozenset({1}): 3,
    frozenset({2}): 3, 
    frozenset({3}): 1,
    frozenset({4}): 2,
    frozenset({1, 2}): 7,
    frozenset({1, 3}): 6,
    frozenset({1, 4}): 7,
    frozenset({2, 3}): 4,
    frozenset({2, 4}): 7,
    frozenset({3, 4}): 3,
    frozenset({1, 2, 3}): 10,
    frozenset({1, 2, 4}): 10,
    frozenset({1, 3, 4}): 9,
    frozenset({2, 3, 4}): 7,
    frozenset({1, 2, 3, 4}): 12
}

value_t_example = {
    frozenset(): 0,
    frozenset({1}): 1,
    frozenset({2}): 1, 
    frozenset({3}): 1,
    frozenset({1, 2}): 3,
    frozenset({1, 3}): 3,
    frozenset({2, 3}): 3,
    frozenset({1, 2, 3}): 4,
}

def v(coalition, value_t) -> int:
    return value_t[frozenset(coalition)]

In [71]:
def check_superadditivity(coalitions, value_t):
    for S in coalitions:
        for T in coalitions:
            if not S & T and not v(S | T, value_t) >= v(S, value_t) + v(T, value_t):
                print(f'Игра не является супераддитивной. S: {S}, T: {T} '
                      f'v(S | T) = {v(S | T, value_t)}, v(S)={v(S, value_t)}, v(T)={v(T, value_t)}')
                return False
    print('Игра является супераддитивной')
    return True


def check_convexity(coalitions, value_t):
    for S in coalitions:
        for T in coalitions:
            if not v(S | T, value_t) + v(S & T, value_t) >= v(S, value_t) + v(T, value_t):
                print(f'Игра не является выпуклой. S: {S}, T: {T} '
                      f'v(S | T) = {v(S | T, value_t)}, v(S & T)={v(S & T, value_t)}, '
                      f'v(S)={v(S, value_t)}, v(T)={v(T, value_t)}')
                return False
    print('Игра является выпуклой')
    return True


def compute_shapley(coalitions, value_t, coalition_members):
    X = []
    N = len(coalition_members)
    for member in coalition_members:
        x_i = 0
        for S in coalitions:
            if member in S:
                #print(f"Coalition S={S} ({v(S, value_t)}) "
                #      f"[S without={S - set([member])}, {v(S - set([member]), value_t)}]")
                x_i += factorial(len(S) - 1) * factorial(N - len(S)) * (v(S, value_t) - v(S - {member}, value_t))
                #print(f"Current x_i: {x_i}")
                
        x_i /= factorial(N)
        #print(f"Final x_i={x_i}")
        X.append(x_i)
    print(f'Вектор Шепли: {[format(x_i, ".3f") for x_i in X]}')

    if np.allclose(sum(X), v(set(coalition_members), value_t)):
        print('Условие групповой рационализации выполняется')
    else:
        print(f'Условие групповой рационализации не выполняется: {sum(X)} and {v(set(coalition_members), value_t)}')

    for member in coalition_members:
        if not X[member - 1] >= v({member}, value_t):
            print(f'Условие индивидуальной рационализации не выполняется '
                  f'(для {member}: (распределённая) {X[member-1]} < {v(set([member]), value_t)} (индивидуальная))')
            break
    else:
        print('Условие индивидуальной рационализации выполняется')


In [72]:
coalition_members = [1, 2, 3, 4]
coalition_members_example = [1, 2, 3]
#coalition_members = coalition_members_example
coalitions = [set(members) for l in range(len(coalition_members)+1) for members in combinations(coalition_members, l)]
#print(f"Coalitions: {coalitions}")
check_superadditivity(coalitions, value_t)
check_convexity(coalitions, value_t)
compute_shapley(coalitions, value_t, coalition_members)

Игра не является супераддитивной. S: {3}, T: {2, 4} v(S | T) = 7, v(S)=1, v(T)=7
Игра не является выпуклой. S: {3}, T: {2, 4} v(S | T) = 7, v(S & T)=0, v(S)=1, v(T)=7
Вектор Шепли: ['4.417', '3.417', '1.583', '2.583']
Условие групповой рационализации выполняется
Условие индивидуальной рационализации выполняется
